In [1]:
import pytesseract

text = pytesseract.image_to_string('/Users/bailey/Downloads/data-science/Vergil-OCR-test/images/forbes-demo-eng.jpg')
print(text)

TesseractError: (1, 'Error, cannot read input file /Users/bailey/Downloads/data-science/Vergil-OCR-test/images/forbes-demo-eng.jpg: No such file or directory Error during processing.')

In [ ]:
import io
import requests
import pytesseract
from PIL import Image
response = requests.get("https://d2jaiao3zdxbzm.cloudfront.net/wp-content/uploads/figure-65.png")
# print( type(response) ) # <class 'requests.models.Response'>
img = Image.open(io.BytesIO(response.content))
# print( type(img) ) # <class 'PIL.JpegImagePlugin.JpegImageFile'>
text = pytesseract.image_to_string(img)
print( text )

ModuleNotFoundError: No module named 'requests'

In [ ]:
import pytesseract
from googletrans import Translator
from PIL import Image

translator = Translator()
im = Image.open("/Users/bailey/Downloads/data-science/Vergil-OCR-test/images/uber-das.png")

text = pytesseract.image_to_string(im, lang = 'deu')
print("ORIGINAL\n",text)

translation = translator.translate(text, dest='en')
print("TRANSLATED\n",text)
print(translation)

ORIGINAL
 über das

TRANSLATED
 über das

Translated(src=de, dest=en, text=about the, pronunciation=None, extra_data="{'confiden...")


Notes:
- doesn't read the scanned version of deu.jpg well at all; reads unscanned ver. of jahrenbericht better than the scanned version..
- doesn't pick up on 'uber des' and some of the smaller text.
- bad at latin
    - better at scanned; cannot decipher V as translation horrible as expected

In [61]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time

xls = pd.read_excel("/Users/bailey/Downloads/PA-vergil-website/Vergil-OCR-test/PAVergilWebsite_BaileyXu_2024.xlsx", 
                    'Website - Draft (Print info.)',
                    header=0, 
                    skiprows=[0])
xls['Date (Print):'] = xls['Date (Print):'].str.replace('.','')
xls['Publisher (IntArc)'] = None
xls['Language (IntArc)'] = None
xls['Contributor (IntArc)'] = None
xls['Date (IntArc)'] = None
xls['DateFLAG (IntArc)'] = False

xls['Publisher (WorldCat)'] = None
xls['Language (WorldCat)'] = None
xls['Contributor (WorldCat)'] = None
xls['Date (WorldCat)'] = None
xls['DateFLAG (WorldCat)'] = False

for i in range (1274, len(xls)):
    title = str(xls["Title/Description (Print – Original):"][i])[:100]
    date = str(xls["Date (Print):"][i])
    if (title=='[–]'):
        continue
        
    out_link = "https://www.google.com/search?q=" + '+'.join(title.split(' '))
    page = requests.get(out_link)
    soup = BeautifulSoup(page.content)
    #print(soup)
    
    links = []
    for link in soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
        for l in re.split(":(?=http)",link["href"].replace("/url?q=","")):
            links.append(l)

    links = [link for link in links if ((link.find("archive.org/details/") > 0) or (link.find("search.worldcat.org") > 0))]
    clean_links = []
    for link in links:
        if (link.find("search.worldcat.org") > 0):
            clean_links.append(link.split("&")[0])
        else:
            clean_links.append(link)
    # print(out_link)
    # print(links)

    if not clean_links:
        print("List is empty")
        continue

    for link in clean_links:
        page = requests.get(link)
        soup = BeautifulSoup(page.content)
        if (link.find("archive.org/details/") > 0):
            # check publication year
            spans = soup.find('span', {'itemprop' : 'datePublished'})

            try:
                link_date = spans.get_text()
                xls.loc[i, 'Date (IntArc)'] = link_date
                if (date!=link_date):
                    xls.loc[i, 'DateFLAG (IntArc)'] = True
                    # print(f"Mismatched date for {title} {link}")
                    # continue
            except:
                pass
            
            # Get Publisher
            try:
                spans = soup.find_all('span', {'itemprop' : 'publisher'})
                publisher = spans[0].get_text()
                # print(publisher)
                xls.loc[i, 'Publisher (IntArc)'] = publisher
            except:
                print(f"Publisher Failed for {title} {link}")
                pass
                
            # Get Language
            try:
                spans = soup.find_all('a',href=True)
                for span in spans:
                    if (span['href'].find("28language%") > 0):
                        lang = span.next
                        # print(lang)
                        xls.loc[i, 'Language (IntArc)'] = lang
                        break
            except:
                print(f"Language Failed for {title} {link}")
                pass
                
            # Get Contributor
            try:
                spans = soup.find('dl', {'class' : 'metadata-definition'})
                if (spans.get_text().find("by") > 0):
                    cont = spans.get_text().strip(' by\n')
                    # print(cont)
                    xls.loc[i, 'Contributor (IntArc)'] = cont
            except:
                print(f"Contributor Failed for {title} {link}")
                pass
            
            # Get pdf link??
        
        elif (link.find("search.worldcat.org") > 0):
            code = link.find("oclc/") + 5
            code = link[code:]
            # print(code)
            
            # check publication year + get language
            try:
                search='format-'+code
                texts = soup.find('span', {'data-testid' : search}).parent
                spans = texts.find_all('span')
                lang = spans[-1].next
    
                texts = str(texts)
                year = re.findall(r'\d+', texts[-12:])
                year = year[0] # str type
                xls.loc[i, 'Date (WorldCat)'] = year

                if (date != year):
                    xls.loc[i, 'DateFLAG (WorldCat)'] = True
                    # print(f"Mismatched date for {title} {link}")
                    # continue
                # print(lang, year)
                xls.loc[i, 'Language (WorldCat)'] = lang
            except:
                print(f"Date/Language Failed for {title} {link}")
                pass

            # Get authors
            try:
                search='author-'+code
                spans = soup.find_all('span', {'aria-labelledby' : search})
                for span in spans:
                    temp = span.find_all('a')
                    cont = ', '.join(l.next for l in temp)
                    # print(cont)
                    xls.loc[i, 'Contributor (WorldCat)'] = cont

            except:
                print(f"Authors Failed for {title} {link}")
                pass

            # Get publisher
            try:
                search='publisher-'+code
                spans = soup.find_all('span', {'data-testid' : search})
                publisher = ''.join(spans[0].text).strip()
                # print(publisher)
                xls.loc[i, 'Publisher (WorldCat)'] = publisher
            except:
                print(f"Publisher Failed for {title} {link}")
                pass

    print(xls.loc[i, ['Publisher (IntArc)','Language (IntArc)','Contributor (IntArc)',
                      'Publisher (WorldCat)','Language (WorldCat)','Contributor (WorldCat)']])
    time.sleep(5)

<>:36: SyntaxWarning: invalid escape sequence '\?'
<>:36: SyntaxWarning: invalid escape sequence '\?'
/var/folders/st/vmdh21yx7z921b5950g1xycw0000gn/T/ipykernel_73735/542074175.py:36: SyntaxWarning: invalid escape sequence '\?'
  for link in soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):


Publisher (IntArc)              Parisiis Apud Hachette
Language (IntArc)                                Latin
Contributor (IntArc)      Rébelliau, Alfred, 1858-1934
Publisher (WorldCat)                              None
Language (WorldCat)                               None
Contributor (WorldCat)                            None
Name: 1274, dtype: object
List is empty
List is empty
List is empty
List is empty
List is empty
List is empty
List is empty
List is empty
List is empty
List is empty
List is empty
List is empty
List is empty
List is empty
List is empty
List is empty
List is empty
Date/Language Failed for Royds, T. F. / The beasts, birds and bees of Virgil . . . handbook to the Georgics. Oxford, Blackwel https://search.worldcat.org/title/1041073314
Publisher Failed for Royds, T. F. / The beasts, birds and bees of Virgil . . . handbook to the Georgics. Oxford, Blackwel https://search.worldcat.org/title/1041073314
Publisher (IntArc)                                   Oxford, B.H. Bl

In [33]:
xls['Publisher (IntArc)'].head(10)

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
Name: Publisher (IntArc), dtype: object

In [ ]:
xls['Language (IntArc)'].head()

In [178]:
xls['Contributor (IntArc)'].head()

0    None
1    None
2    None
3    None
4    None
Name: Contributor (IntArc), dtype: object

In [ ]:
xls['Publisher (WorldCat)'].head()

0    ex Chalcographia R.G.A. apud pedem marmoreum, ...
1                                                 None
2                                                 None
3                                                 None
4                                                 None
Name: Publisher (WorldCat), dtype: object

In [48]:
xls['Language (WorldCat)'].head()

0    None
1    None
2    None
3    None
4    None
Name: Language (WorldCat), dtype: object

In [179]:
xls['Contributor (WorldCat)'].head(10)

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
8    None
9    None
Name: Contributor (WorldCat), dtype: object

In [ ]:
print(xls.loc[0, ['Publisher (IntArc)','Language (IntArc)','Contributor (IntArc)',
                      'Publisher (WorldCat)','Language (WorldCat)','Contributor (WorldCat)']])

<bound method Series.to_frame of Publisher (IntArc)             ex Chalcographia R.G.A. apud pedem marmoreum
Language (IntArc)                                                     Latin
Contributor (IntArc)                                 Publius Vergilius Maro
Publisher (WorldCat)      ex Chalcographia R.G.A. apud pedem marmoreum, ...
Language (WorldCat)                                                   Latin
Contributor (WorldCat)                     Wergiliusz, Pietro Santi Bartoli
Name: 0, dtype: object>


In [11]:
import pandas as pd
import numpy as np

xls = pd.read_excel('/Users/bailey/Downloads/Vergil.xlsx',
                    header=0)
for i in range (len(xls)):
    if (xls['Title/Description (Print – Original):'][i]=='+++++'):
        xls.loc[i, "Author (Modern):"] = np.NaN
        xls.loc[i, "Location Code:"] = np.NaN
        xls.loc[i, "Location:"] = np.NaN
        xls.loc[i, "Language:"] = np.NaN
        xls.loc[i, "Date (Print):"] = np.NaN

xls.head(30)

,Unnamed: 0,Entry:,Entry (Print):,Location:,Location Code:,Call Number:,Barcodes:,Category (Print):,Title/Description (Print – Original):,Title/Description (Print – Translated):,...,Publisher (IntArc),Language (IntArc),Contributor (IntArc),Date (IntArc),DateFLAG (IntArc),Publisher (WorldCat),Language (WorldCat),Contributor (WorldCat),Date (WorldCat),DateFLAG (WorldCat)
0,0,1,1.0,Special Collections (in Storage),Sp Col LAT,873 V5x 1741c,PAN-5607072,Printed Copies of Codices,Antiquissimi Virgiliani Codicis fragmenta et p...,NaN,...,ex Chalcographia R.C.A. apud pedem marmoreum,Latin,Publius Vergilius Maro,1741,0.0,"ex Chalcographia R.G.A. apud pedem marmoreum, ...",Latin,"Publius Vergilius Maro, Pietro Santi Bartoli",1741.0,0.0
1,1,2,2.0,Special Collections (in Storage),Sp Col LAT,873 V5X 1741,PAN-5605500,Printed Copies of Codices,Codex Mediceus. P. V. M. Codex antiquissimus a...,NaN,...,typis Mannianis,Latin,Publius Vergilius Maro,1741,0.0,NaN,NaN,NaN,NaN,0.0
2,2,3,3.0,Special Collections (in Storage),Sp Col LAT,873 V5x 1835,PAN-5607076,Printed Copies of Codices,Homeri Iliados picturae antiquae ex codice Med...,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
3,3,4,4.0,Special Collections (in Storage),Sp Col LAT,873 V5x 1930b,PAN-5607078,Printed Copies of Codices,Francisci Petrarcae Vergilianus Codex. Ad Publ...,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
4,4,5,5.0,NaN,NaN,NaN,NaN,Printed Copies of Codices,"Crous, E. Vergilausgaben grosser Drucker. Berg...",NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
5,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
6,6,6,6.0,Special Collections (in Storage),Sp Col LAT,873 V5x 1476,PAN-5607037,Editions (Incunabula),Begins: Maronis Vita. Commentary surrounds tex...,NaN,...,NaN,Latin,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
7,7,7,7.0,NaN,NaN,NaN,NaN,Editions (Incunabula),Bucolica (in Italian). Prefatione di Bernardo ...,NaN,...,NaN,Italian,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
8,8,8,8.0,Special Collections (in Storage),Sp Col LAT,873 V5x 1492,97602-1001,Editions (Incunabula),Colophon: P. V. M. opera: cum Seruii . . . A. ...,NaN,...,NaN,Latin,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
9,9,9,9.0,Special Collections (in Storage),Sp Col LAT,873 V5x 1493,PAN-5612514,Editions (Incunabula),Vergilius cum cōmentariis quinque videlicet: S...,NaN,...,"Per Bartolomeu[m] de Zanis de Portesio, Bartho...",Latin,"Publius Vergilius Maro, Servius, Cristoforo La...",1493,0.0,NaN,NaN,NaN,NaN,0.0


In [12]:
# determining the name of the file
file_name = 'vergil_ghjkgh.xlsx'
 
# saving the excel
xls.to_excel(file_name)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.
